In [1]:
!pip install tensorflow
import tensorflow as tf
print("Versão do TensorFlow:", tf.__version__)


Versão do TensorFlow: 2.15.0


In [2]:
import keras as K
print("Versão do Keras:", K.__version__)

Versão do Keras: 2.15.0


In [3]:
# Imports
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

In [4]:
# Inicializando a Rede Neural Convolucional
classifier = Sequential()

# Passo 1 - Primeira Camada de Convolução
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))

# Passo 2 - Pooling
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Adicionando a Segunda Camada de Convolução
classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Adicionando a Terceira Camada de Convolução
classifier.add(Conv2D(128, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Passo 3 - Flattening
classifier.add(Flatten())

# Passo 4 - Full connection
classifier.add(Dense(units=256, activation='relu'))
classifier.add(Dropout(0.5))  # Adicionando dropout para regularização
classifier.add(Dense(units=1, activation='sigmoid'))

# Compilando a rede
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [5]:
# Criando os objetos train_datagen e validation_datagen com as regras de pré-processamento das imagens
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

validation_datagen = ImageDataGenerator(rescale = 1./255)

In [6]:
# Pré-processamento das imagens de treino e validação
training_set = train_datagen.flow_from_directory('training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

validation_set = validation_datagen.flow_from_directory('test_set',
                                                        target_size = (64, 64),
                                                        batch_size = 32,
                                                        class_mode = 'binary')

Found 196 images belonging to 2 classes.


Found 79 images belonging to 2 classes.


In [7]:
# Executando o treinamento (pode levar bastante tempo, dependendo do seu computador)
classifier.fit_generator(training_set,
                         steps_per_epoch=8000,
                         epochs=5,
                         validation_data=validation_set,
                         validation_steps=2000)


Epoch 1/5



C:\Users\rafae\AppData\Local\Temp\ipykernel_23576\1029423535.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(training_set,



8000/8000 [==============================] - 1s 67us/step - loss: 0.7155 - accuracy: 0.5306 - val_loss: 0.6798 - val_accuracy: 0.5823


In [8]:
# Primeira Imagem
import PIL
import numpy as np
from keras.preprocessing import image

path = 'test_set/bart/bart27.bmp'
img = PIL.Image.open(path)
img.save(path.replace('.bmp', '.png'))


test_image = image.load_img(path.replace('.bmp', '.png'), target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices

if result[0][0] == 1:
    prediction = 'Bart'
else:
    prediction = 'Homer'


1/1 [==============================] - 0s 66ms/step


In [9]:
# Previsão da primeira imagem
prediction

'Homer'

In [10]:
# Avaliação do modelo no conjunto de teste
result = classifier.evaluate(validation_set, steps=2000)
print("Loss:", result[0])
print("Accuracy:", result[1])


2000/2000 [==============================] - 0s 17us/step - loss: 0.6798 - accuracy: 0.5823
Loss: 0.6797916889190674
Accuracy: 0.5822784900665283
